# Este archivo contiene el código necesario para poder leer los ficheros xml y txt.Debido a que este contenido aún no se ha visto en clase, pero se verá a lo largo del módulo 1

## Lectura del fichero XML

Nos devuelve una lista que contiene diccionarios con los diferentes elementos del archivo xml.

Ejemplo:

[{'level_0': 1, 'index': 1'time': 784, 'age': '50-54', 'gender': '0' },
 {'level_0': 2, 'index': 2'time': 924, 'age': '22-24', 'gender': '0'},
 {'level_0': 2, 'index': 2'time': 575, 'age': '45-49', 'gender': '0'},
 {'level_0': 2, 'index': 2'time': 781, 'age': '45-49', 'gender': '0'}
 {'level_0': 2, 'index': 2'time': 1020, 'age': '25-29', 'gender': '1'}]


> Recomendación: Aprender a tratar los datos de un elemento del diccionario, antes de intentar aplicarlo a todos los datos del fichero xml.

In [1]:
import re
import os
import xml.etree.ElementTree as ET
import mysql.connector

In [8]:
class Archivo_Xml:
    def __init__(self,ruta):
        self.ruta = ruta
        
    def apertura_xml(self,archivo_xml):
    
        tree = ET.parse(self.ruta+archivo_xml)
        root = tree.getroot()

        xml_list = []
        for child in root:
            xml_dict = {}
            for subchild in child:
                xml_dict[subchild.tag] = subchild.text
            xml_list.append(xml_dict)
        
        return xml_list

    def procesado_xml(self,archivo_xml):
        for fila in xml_list:
            if fila['gender'] == '0':
                fila['gender'] = 'Man'
            elif fila['gender'] == '1':
                fila['gender'] = 'Woman'
            elif fila['gender'] == '2':
                fila['gender'] = 'Nonbinary'
            elif fila['gender'] == '3':
                fila['gender'] = 'Prefer not to say'
            elif fila['gender'] == '4':
                fila['gender'] = 'Prefer to self-describe'

        for fila in xml_list:
            if 'level_0' in fila:
                fila.pop('level_0')
            else:
                pass
            
        return xml_list
    
    def creacion_tabla_xml(self,archivo_xml):
        lista_tupla = []
        for linea in archivo_xml:
            lista_tupla.append(tuple(linea.values()))
            
        return lista_tupla  
    
    def insercion_datos_xml(self):
        cnx = mysql.connector.connect(user='root', password='AlumnaAdalab', host='127.0.0.1', database='adalab')

        mycursor = cnx.cursor()

        sql = """INSERT INTO xml (index_xml, time, age, gender)
                VALUES (%s, %s, %s, %s)"""
        val = self.creacion_tabla_xml(xml_list)

        try: 
            mycursor.executemany(sql, val)
            cnx.commit()
            print(mycursor.rowcount, "registro/s insertado/s.")

        except mysql.connector.Error as err:
            print(err)
            print("Error Code:", err.errno)
            print("SQLSTATE", err.sqlstate)
            print("Message", err.msg)

In [9]:
archivo_xml = Archivo_Xml('data\\')

In [10]:
xml_list = archivo_xml.apertura_xml('data_xml.xml')

In [11]:
xml_list = archivo_xml.procesado_xml(xml_list)

In [12]:
resultado_xml = archivo_xml.creacion_tabla_xml(xml_list)

In [13]:
archivo_xml.insercion_datos_xml()

25972 registro/s insertado/s.


## Lectura del fichero TXT

Nos devuelve una lista que contiene todos los elementos del archivo txt, cada elemento de la lista se toma como un único string. Cada elemento de la lista termina con \n

Ejemplo:

['index_sql;Q3;Q4;Q5;Q6;Q7;Q8;Q9;Q11;Q12;Q13;Q14;Q15;Q16;Q17;Q20;Q21;Q22;Q23;Q24;Q25;Q26;Q32;Q33;Q34;Q35;Q41\n',
 '1;Indonesia;Master’s degree;Program/Project Manager;20+ years;null, SQL, C, C++, Java;Python;null,   Notepad++  ,  Jupyter Notebook;A cloud computing platform (AWS, Azure, GCP, hosted notebooks, etc);null, None;Never; Matplotlib ;Under 1 year;  Scikit-learn ;Linear or Logistic Regression, Decision Trees or Random Forests;Manufacturing/Fabrication;1000-9,999 employees;1-2;We are exploring ML methods (and may one day put a model into production);null, Build and/or run the data infrastructure that my business uses for storing, analyzing, and operationalizing data;60,000-69,999;$0 ($USD);;;;;Advanced statistical software (SPSS, SAS, etc.)\n',
 '2;Pakistan;Master’s degree;Software Engineer;1-3 years;Python, C++, Java;Python;null,  PyCharm ,  Jupyter Notebook, Other;A laptop;null, Other;Never; Matplotlib ;I do not use machine learning methods;;;Academics/Education;1000-9,999 employees;0;I do not know;null, None of these activities are an important part of my role at work;$0-999;$0 ($USD);MySQL , MongoDB ;MySQL ;null, None;;Basic statistical software (Microsoft Excel, Google Sheets, etc.)\n',
 '3;Mexico;Doctoral degree;Research Scientist;20+ years;Python;Python;null,   Spyder  ,  Jupyter Notebook;A cloud computing platform (AWS, Azure, GCP, hosted notebooks, etc); NVIDIA GPUs ;More than 25 times; Matplotlib ;5-10 years;  Scikit-learn ,   TensorFlow ,  Keras ;null, Dense Neural Networks (MLPs, etc), Convolutional Neural Networks, Recurrent Neural Networks;Academics/Education;1000-9,999 employees;0;I do not know;null, Do research that advances the state of the art of machine learning;30,000-39,999;$0 ($USD);;;;;Local development environments (RStudio, JupyterLab, etc.)\n',


> Recomendación: Aprender a tratar los datos de algunos de los elementos de la lista, antes de intentar aplicarlo a todos los datos del fichero txt.
 - Primer elemento de la lista:
    lista[0] = 'index_sql;Q3;Q4;Q5;Q6;Q7;Q8;Q9;Q11;Q12;Q13;Q14;Q15;Q16;Q17;Q20;Q21;Q22;Q23;Q24;Q25;Q26;Q32;Q33;Q34;Q35;Q41\n'

In [54]:
class Archivo_Txt:
    def __init__(self,ruta):
        self.ruta = ruta
    
    def apertura_txt(self, archivo_txt):
        with open(self.ruta+archivo_txt,'r') as file: #self.ruta/arch                                                     ivo_txt
            file_source = file.readlines()
        return file_source
    
    def procesado_txt(self, archivo_txt):
        lista_source = [archivo_txt]
        file_limpio = []

        for lista in lista_source:
            for campo in lista:
                file_limpio.append(campo.replace("\n", "").replace("null", "NULL").replace("<","under").replace("   "," ").replace("  "," ").replace(" ,",",").strip())
        
        return file_limpio
    
    def creacion_tabla_txt(self, archivo_txt):
        lista_tupla = []
        for linea in archivo_txt:
            lista_tupla.append(tuple(linea.split(";")))
            
        return lista_tupla
    
    def insercion_datos(self):
        cnx = mysql.connector.connect(user='root', password='AlumnaAdalab', host='127.0.0.1', database='adalab')

        mycursor = cnx.cursor()

        sql = """INSERT INTO txt (index_txt, q3, q4, q5, q6, q7, q8, q9, q11, q12, q13, q14, q15, q16, q17, 
                                q20, q21, q22, q23, q24, q25, q26, q32, q33, q34, q35, q41)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""
        val = self.creacion_tabla_txt(file_source)[1:]

        try: 
            mycursor.executemany(sql, val)
            cnx.commit()
            print(mycursor.rowcount, "registro/s insertado/s.")

        except mysql.connector.Error as err:
            print(err)
            print("Error Code:", err.errno)
            print("SQLSTATE", err.sqlstate)
            print("Message", err.msg)

In [55]:
archivo_txt = Archivo_Txt('data\\')

In [56]:
file_source = archivo_txt.apertura_txt('data_txt.txt')

In [57]:
file_source = archivo_txt.procesado_txt(file_source)

In [58]:
resultado_txt = archivo_txt.creacion_tabla_txt(file_source)[1:]

In [59]:
archivo_txt.insercion_datos()

25972 registro/s insertado/s.
